In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df=pd.read_csv('zomato.csv')
df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
feature_na=[feature for feature in df.columns if df[feature].isnull().sum()>0]
feature_na

In [ ]:
#% of missing values
for feature in feature_na:
    print('{} has {} % missing values'.format(feature,np.round(df[feature].isnull().sum()/len(df)*100,4)))

In [ ]:
df['rate'].unique()

In [ ]:
df.dropna(axis = 'index',subset=['rate'],inplace=True)

In [ ]:
def split(x):
    return x.split('/')[0]

In [ ]:
df['rate']=df['rate'].apply(split)
df.head()

In [ ]:
df['rate'].unique()

In [ ]:
df.replace('NEW',0,inplace=True)

In [ ]:
df.replace('-',0,inplace=True)

In [ ]:
df['rate']=df['rate'].astype(float)

### calculate avg rating of each resturant

In [ ]:
df.groupby('name')['rate'].mean().nlargest(20).plot.bar()
plt.show()

In [ ]:
df_rate=df.groupby('name')['rate'].mean().to_frame()
df_rate=df_rate.reset_index()
df_rate.columns=['restaurant','rating']
df_rate.head(20)

In [ ]:
df_rate.shape

In [ ]:
# Rating distribution
sns.set_style(style='whitegrid')
sns.distplot(df_rate['rating'])
plt.show()

#### Which are the top restaurant chains in Bangaluru?

In [ ]:
plt.figure(figsize=(10,7))
chains=df['name'].value_counts()[0:20]
sns.barplot(x=chains,y=chains.index,palette='deep')
plt.title("Most famous restaurants chains in Bangaluru")
plt.xlabel("Number of outlets") 
plt.show()

#### How many of the restuarants do not accept online orders?

In [ ]:
x=df["online_order"].value_counts()
x

In [ ]:
labels=['accepted','Not_accepted']

In [ ]:
import plotly.express as px

In [ ]:
px.pie(df,values=x,labels=labels,title='Pie chart')

#### What is the ratio b/w restaurants that provide and do not provide table booking ?

In [ ]:
x=df['book_table'].value_counts()
labels=['not book','book']

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
trace=go.Pie(labels=labels,values=x,hoverinfo='label+percent',textinfo='value')
iplot([trace])

#### How many types of restaurants we have?

In [ ]:
df['rest_type'].isna().sum()

In [ ]:
df['rest_type'].dropna(inplace=True)

In [ ]:
len(df['rest_type'].unique())

In [ ]:
trace1=go.Bar(x=df['rest_type'].value_counts().nlargest(20).index,
      y=df['rest_type'].value_counts().nlargest(20)
      )

In [ ]:
iplot([trace1])

#### highest voted restaurant

In [ ]:
df.groupby('name')['votes'].sum().nlargest(20).plot.bar()
plt.show()

In [ ]:
trace1=go.Bar(x=df.groupby('name')['votes'].sum().nlargest(20).index,
       y=df.groupby('name')['votes'].sum().nlargest(20))
iplot([trace1])


### total restaurants at different locations of Bangalore

In [ ]:
restaurant=[]
location=[]
for key,location_df in df.groupby('location'):
    location.append(key)
    restaurant.append(len(location_df['name'].unique()))

In [ ]:
df_total=pd.DataFrame(zip(location,restaurant))
df_total.columns=['location','restaurant']
df_total.head()

In [ ]:
df_total.set_index('location',inplace=True)
df_total.head()

In [ ]:
df_total.sort_values(by='restaurant').tail(10).plot.bar()
plt.show()

### Total number of variety of restaurants ie north indian,south Indian

In [ ]:
cuisines=df['cuisines'].value_counts()[0:10]
cuisines

In [ ]:
trace1=go.Bar(
x=df['cuisines'].value_counts()[0:10].index,
y=df['cuisines'].value_counts()[0:10]
)

In [ ]:
iplot([trace1])

In [ ]:
df.columns

### Analyse Approx cost for 2 people

In [ ]:
df['approx_cost(for two people)'].isna().sum()

In [ ]:
df.dropna(axis='index',subset=['approx_cost(for two people)'],inplace=True)

In [ ]:
df['approx_cost(for two people)'].unique()

In [ ]:
df['approx_cost(for two people)']=df['approx_cost(for two people)'].apply(lambda x:x.replace(',',''))
df['approx_cost(for two people)'].unique()

In [ ]:
df['approx_cost(for two people)']=df['approx_cost(for two people)'].astype(int)

In [ ]:
df['approx_cost(for two people)'].dtype

In [ ]:
sns.distplot(df['approx_cost(for two people)'])

#### cost vs rating

In [ ]:
sns.scatterplot(x='rate',y='approx_cost(for two people)',hue='online_order',data=df)

### Is there any difference b/w votes of restaurants accepting and not accepting online orders?

In [ ]:
sns.boxplot(x='online_order',y='votes',data=df)

In [ ]:
px.box(df,x='online_order',y='votes')

### Is there any difference b/w price of restaurants accepting and not accepting online orders?

In [ ]:
px.box(df,x='online_order',y='approx_cost(for two people)')

### Cheapest Rate for 2 people

In [ ]:
df['approx_cost(for two people)'].min()

### Most costly for 2 people

In [ ]:
df['approx_cost(for two people)'].max()

In [ ]:
df[df['approx_cost(for two people)']==6000]['name']

In [ ]:
data=df.copy()

In [ ]:
data.set_index('name',inplace=True)

In [ ]:
data.head()

### Most costly Rate for 2 people is served at which Restaurant what exactly is the dish involved in this and liked dish of that restaurant

In [ ]:
df[df['approx_cost(for two people)']==6000][['name','cuisines','dish_liked']]

### Top 10 Most Expensive restaurant with approx cost for 2 people

In [ ]:
data['approx_cost(for two people)'].nlargest(10).plot.bar()

In [ ]:
trace1 = go.Bar( 
        x = data['approx_cost(for two people)'].nlargest(10).index,
        y = data['approx_cost(for two people)'].nlargest(10),
        name= 'Priority')
iplot([trace1])

### Top 10 Cheapest restaurant with approx cost for 2 people

In [ ]:
data['approx_cost(for two people)'].nsmallest(10).plot.bar()

In [ ]:
trace1 = go.Bar( 
        x = data['approx_cost(for two people)'].nsmallest(10).index,
        y = data['approx_cost(for two people)'].nsmallest(10),
        name= 'Priority')
iplot([trace1])

### all the restautant that are below than 500(budget hotel)

In [ ]:
data[data['approx_cost(for two people)']<=500]

In [ ]:
df_budget=data[data['approx_cost(for two people)']<=500].loc[:,('approx_cost(for two people)')]
df_budget=df_budget.reset_index()
df_budget.head()

### Restaurants that have better rating >4 and that are under budget too

In [ ]:
df[(df['rate']>=4) & (df['approx_cost(for two people)']<=500)].shape


### Total no. of Restaurants that have better rating >4 and that are under budget too ie less than 500

In [ ]:
df_new=df[(df['rate']>=4) & (df['approx_cost(for two people)']<=500)]
len(df_new['name'].unique())

In [ ]:
df_new.head()

### Total such various affordable hotels at different location

In [ ]:
location=[]
total=[]

for loc,location_df in df_new.groupby('location'):
    location.append(loc)
    total.append(len(location_df['name'].unique()))

In [ ]:
location_df=pd.DataFrame(zip(location,total))
location_df.head()

In [ ]:
location_df.columns=['location','restaurant']
location_df.head()

### Finding Best budget Restaurants in any location

In [ ]:
def return_budget(location,restaurant):
    budget=df[(df['approx_cost(for two people)']<=400) & (df['location']==location) & 
                     (df['rate']>4) & (df['rest_type']==restaurant)]
    return(budget['name'].unique())

In [ ]:
return_budget('BTM',"Quick Bites")

### Which are the foodie areas?

In [ ]:
plt.figure(figsize=(10,7))
Restaurant_locations=df['location'].value_counts()[:20]
sns.barplot(Restaurant_locations,Restaurant_locations.index)

In [ ]:
Restaurant_locations=df['location'].value_counts()[:20]
trace1 = go.Bar( 
        x = Restaurant_locations.index,
        y = Restaurant_locations,
        name= 'Priority')
iplot([trace1])

### Geographical Analysis

In [ ]:
locations=pd.DataFrame({'Name':df['location'].unique()})
locations.head()

In [ ]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator=Nominatim(user_agent='app')

In [ ]:
lat_lon=[]
for location in locations['Name']:
    location=geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

In [ ]:
locations['geo_loc']=lat_lon

In [ ]:
locations.head()

In [ ]:
locations.shape

In [ ]:
Rest_locations=pd.DataFrame(df['location'].value_counts().reset_index())
Rest_locations.head()

In [ ]:
Rest_locations.columns=['Name','count']
Rest_locations.head()

In [ ]:
Restaurant_locations=Rest_locations.merge(locations,on='Name',how='left').dropna()
Restaurant_locations.head()

In [ ]:
np.array(Restaurant_locations['geo_loc'])

In [ ]:
lat,lon=zip(*np.array(Restaurant_locations['geo_loc']))

In [ ]:
type(lat)

In [ ]:
Restaurant_locations['lat']=lat
Restaurant_locations['lon']=lon

In [ ]:
Restaurant_locations.head()

In [ ]:
Restaurant_locations.drop('geo_loc',axis=1,inplace=True)
Restaurant_locations.head()

In [ ]:
import folium
from folium.plugins import HeatMap

In [ ]:
def generatebasemap(default_location=[13.29,77.75],default_zoom_start=12):
    basemap=folium.Map(location=default_location,zoom_start=default_zoom_start)
    return basemap

In [ ]:
basemap=generatebasemap()

In [ ]:
basemap

In [ ]:
HeatMap(Restaurant_locations[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)

In [ ]:
basemap

### Heatmap of North Indian restaurants

In [ ]:
df.head()

In [ ]:
df2=df[df['cuisines']=='North Indian']
df2.head()

In [ ]:
north_india=df2.groupby(['location'],as_index=False)['url'].agg('count')
north_india.columns=['Name','count']
north_india.head()

In [ ]:
north_india=north_india.merge(locations,on="Name",how='left').dropna()

In [ ]:
north_india.head(10)

In [ ]:
north_india['lat'],north_india['lon']=zip(*north_india['geo_loc'].values)
north_india.head()

In [ ]:
north_india.drop('geo_loc',axis=1,inplace=True)
north_india.head()

In [ ]:
basemap=generatebasemap()
HeatMap(north_india[['lat','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

In [ ]:
df3= df[df['cuisines']=='South Indian']
south_india=df2.groupby(['location'],as_index=False)['url'].agg('count')
south_india.columns=['Name','count']
south_india=south_india.merge(locations,on="Name",how='left').dropna()
south_india['lan'],south_india['lon']=zip(*south_india['geo_loc'].values)

In [ ]:
south_india=south_india.drop(['geo_loc'],axis=1)
south_india.head()

In [ ]:
basemap=generatebasemap()
HeatMap(south_india[['lan','lon','count']].values.tolist(),zoom=20,radius=15).add_to(basemap)
basemap

In [ ]:
df_1=df.groupby(['rest_type','name']).agg('count')
df_1

In [ ]:
datas=df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False).head(3))['url'].reset_index().rename(columns={'url':'count'})

In [ ]:
datas

In [ ]:
df_1=df.groupby(['rest_type','name']).agg('count')
df_1

In [ ]:
df_1.sort_values(['url'],ascending=False)

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url']

In [ ]:
df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url'].reset_index()

In [ ]:
dataset=df_1.sort_values(['url'],ascending=False).groupby(['rest_type'],
                as_index=False).apply(lambda x : x.sort_values(by="url",ascending=False))['url'].reset_index().rename(columns={'url':'count'})

In [ ]:
dataset

In [ ]:
casual=dataset[dataset['rest_type']=='Casual Dining']
casual